In [ ]:
import numpy as np
import panel as pn

PRIMARY_COLOR = "#0072B5"
SECONDARY_COLOR = "#B54300"

pn.extension(design="material", sizing_mode="stretch_width")

In [ ]:
from settlements import parse_settlements
from spatial_sim import Params, init_state, step_state

In [ ]:
params = Params(
    beta=32, seasonality=0.16, demog_scale=1.0, 
    mixing_scale=0.002, distance_exponent=1.5)

In [ ]:
@pn.cache
def get_data():
    return parse_settlements()

settlements_df = get_data()

settlements_df.head()

In [ ]:
state = init_state(settlements_df, params)
t = 0

In [ ]:
beta_slider = pn.widgets.FloatSlider(value=params.beta, start=0, end=50, step=1, name='beta')
def on_beta_change(value):
    params.beta = value
bound_beta = pn.bind(on_beta_change, value=beta_slider)

seasonality_slider = pn.widgets.FloatSlider(value=params.seasonality, start=0, end=0.3, step=0.02, name='seasonality')
def on_seasonality_change(value):
    params.seasonality = value
bound_seasonality = pn.bind(on_seasonality_change, value=seasonality_slider)

demog_scale_slider = pn.widgets.FloatSlider(value=params.demog_scale, start=0.1, end=1.5, step=0.05, name='demog_scale')
def on_demog_scale_change(value):
    params.demog_scale = value
    params.biweek_avg_births = params.demog_scale * params.births / 26.
    params.biweek_death_prob = params.demog_scale * params.births / params.population / 26.
bound_demog_scale = pn.bind(on_demog_scale_change, value=demog_scale_slider)

In [ ]:
from bokeh import models, plotting, io
from bokeh.palettes import Reds256

source = models.ColumnDataSource(dict(
    name=settlements_df.index,
    x=settlements_df.Long, 
    y=settlements_df.Lat,
    size=0.03*np.sqrt(settlements_df.population),
    prevalence=state[:, 1] / state[:, :].sum(axis=-1)
))

exp_cmap = models.LogColorMapper(palette=Reds256[::-1], low=1e-4, high=0.01)

p = plotting.figure(
    x_axis_label="Longitude", y_axis_label="Latitude",
    title="Prevalence",
)
p.scatter(x="x", y="y", size="size", color={"field": "prevalence", "transform": exp_cmap}, source=source, alpha=0.5)

io.curdoc().add_root(p)

def simulate_step():
    global t
    t += 1
    yield step_state(state, params, t)

def stream():
    next(simulate_step())
    p.title.text = "Prevalence ({:.2f} years)".format(t/26.)
    source.data["prevalence"] = state[:, 1] / state[:, :].sum(axis=-1)

io.curdoc().add_periodic_callback(stream, 50)

In [ ]:
pn.Column(
    pn.Row(beta_slider, bound_beta),
    pn.Row(seasonality_slider, bound_seasonality),
    pn.Row(demog_scale_slider, bound_demog_scale)
)

In [ ]:
pn.template.MaterialTemplate(
    site="numpy demo",
    title="Interactive Spatial Simulation",
    sidebar=[beta_slider, seasonality_slider, demog_scale_slider],
    main=[p],
).servable();  # The ; is needed in the notebook to not display the template

In [ ]:
# In a terminal run: panel serve interactive.ipynb --autoreload
# Navigate to served site at: http://localhost:[port]/interactive